# Code Interpreting with Gemini 1.5 Pro/Flash
This example uses the E2B's [Code Interpreter](https://github.com/e2b-dev/code-interpreter) as a tool for Gemini 1.5 pro/flash. We ask the LLM to show a chart which means the LLM will generate Python code that will get sent to E2B, we display the chart.

This example is using Google AI Studio.

# Step 1: Install the dependencies


In [47]:
# Install the required packages
%pip install google-generativeai e2b_code_interpreter --upgrade

  Using cached google_generativeai-0.6.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached e2b_code_interpreter-0.0.8-py3-none-any.whl.metadata (3.2 kB)
  Using cached e2b-0.17.1-py3-none-any.whl.metadata (3.2 kB)
Using cached google_generativeai-0.6.0-py3-none-any.whl (158 kB)
Using cached e2b_code_interpreter-0.0.8-py3-none-any.whl (9.9 kB)
Using cached e2b-0.17.1-py3-none-any.whl (102 kB)
  Attempting uninstall: e2b
    Found existing installation: e2b 0.17.0
    Uninstalling e2b-0.17.0:
      Successfully uninstalled e2b-0.17.0
  Attempting uninstall: e2b_code_interpreter
    Found existing installation: e2b-code-interpreter 0.0.6
    Uninstalling e2b-code-interpreter-0.0.6:
      Successfully uninstalled e2b-code-interpreter-0.0.6
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.5.4
    Uninstalling google-generativeai-0.5.4:
      Successfully uninstalled google-generativeai-0.5.4
Note: you may need to restart the kernel to use up

# Step 2: Define API keys, prompts and the tool

Get your API keys:

- E2B API key: https://e2b.dev/docs
- Google Gemini API key: https://aistudio.google.com/app/apikey

In [48]:
# Get your API keys or save them to .env file.
import os
from dotenv import load_dotenv
load_dotenv()

# TODO: Get your Google AI Studio API key. See here for more details: https://ai.google.dev/gemini-api/docs/api-key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# TODO: Get your E2B API key from https://e2b.dev/docs
E2B_API_KEY = os.getenv("E2B_API_KEY")

# TODO: Set your model name
MODEL_NAME = "gemini-1.5-flash-latest" # other options: "gemini-1.5-pro-latest"

# System prompt for the LLM
SYSTEM_PROMPT = """
## your job & context
you are a python data scientist. you are given tasks to complete and you run python code to solve them.
You DO NOT MAKE SYNTAX MISTAKES OR FORGET ANY IMPORTS
- the python code runs in jupyter notebook.
- every time you call `execute_python_code` tool, the python code is executed in a separate cell. it's okay to multiple calls to `execute_python_code`.
- display visualizations using matplotlib or any other visualization library directly in the notebook. don't worry about saving the visualizations to a file.
- you have access to the internet and can make api requests.
- you also have access to the filesystem and can read/write files.
- you can install any pip package (if it exists) if you need to but the usual packages for data analysis are already preinstalled.
- you can run any python code you want, everything is running in a secure sandbox environment.
"""

Function calling is defined differently in Google's Generative AI SDK. Instead of defining the function schema in JSON, you can directly provide a Python function. The SDK will automatically convert the function into a format the API understands.

More details here: https://ai.google.dev/gemini-api/docs/function-calling/tutorial?lang=python#basics_of_function_calling

In [49]:
# def execute_python_code(code: str):
#     """
#     Execute python code in a Jupyter notebook cell and returns any result, stdout, stderr, display_data, and error

#     Args:
#     - code: str: The python code to execute in a single cell.
#     """
#     print("Running code interpreter...")
#     print(f"Code to execute: \n```\n{code}\n```")
#     exec = e2b_code_interpreter.notebook.exec_cell(
#         code,
#         on_stderr=lambda stderr: print("[Code Interpreter]", stderr),
#         on_stdout=lambda stdout: print("[Code Interpreter]", stdout),
#         )
#     if exec.error:
#        print("[Code Interpreter ERROR]", exec.error)
#     else:
#       return exec.results
    

def execute_python_code(e2b_code_interpreter, code):
  print("Running code interpreter...")
  exec = e2b_code_interpreter.notebook.exec_cell(
    code,
    on_stderr=lambda stderr: print("[Code Interpreter]", stderr),
    on_stdout=lambda stdout: print("[Code Interpreter]", stdout),
    # You can also stream code execution results
    # on_result=...
  )

  if exec.error:
    print("[Code Interpreter ERROR]", exec.error)
  else:
    return exec.results

# Step 2: Initialise the LLM

In [50]:
import google.generativeai as genai

# Configure the API key
genai.configure(api_key=GOOGLE_API_KEY)

# Initialize the model, and pass the system instruction and tools
model = genai.GenerativeModel(model_name=MODEL_NAME, 
                              system_instruction=SYSTEM_PROMPT,
                              tools=[execute_python_code])

# Step 3: Run the LLM, extract the tool arguments, and execute the tool

In [51]:
from e2b_code_interpreter import CodeInterpreter

with CodeInterpreter(api_key=E2B_API_KEY) as code_interpreter:
    # Send the prompt to the model
    response = model.generate_content("Visualize a distribution of height of men based on the latest data you know.")

    # Extract the arguments (i.e. `code`) from tool call
    # More details here: https://ai.google.dev/gemini-api/docs/function-calling/tutorial?lang=python#optional_low_level_access
    python_code = response.candidates[0].content.parts[0].function_call.args["code"]

    # Execute the code
    results = execute_python_code(python_code)

    if results:
        first_result = results[0]
    else:
        print("No code results")
        exit(0)

# Display the first result
# You can also access the data directly
# first_result.png
# first_result.jpg
# first_result.pdf
# ...
first_result

InvalidArgument: 400 * GenerateContentRequest.tools[0].function_declarations[0].parameters.properties[e2b_code_interpreter].type: field predicate failed: $ != TYPE_UNSPECIFIED
* GenerateContentRequest.tools[0].function_declarations[0].parameters.properties[code].type: field predicate failed: $ != TYPE_UNSPECIFIED
